In [1]:
!which python


/Users/saiha/opt/anaconda3/envs/churn-model/bin/python


In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
#standard imports
import os
import pickle
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,cross_val_score,learning_curve
import sklearn

from src.localpath import *
from src.data.make_dataset import load_training_data
from src.models.train_model import *
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
AdaBoostClassifier,
GradientBoostingClassifier,
RandomForestClassifier,


)
from sklearn.inspection import plot_partial_dependence
from sklearn.metrics import roc_curve

In [4]:
#Configuration for Jupyter Notebooks
pd.options.display.max_columns=100
pd.options.display.max_rows=100

## Load Featurized  Data

In [29]:
X_train,y_train=load_training_data(final=True)

# Feature Multicolinearity

In [17]:
vifs=[variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]


In [18]:
sorted(zip (X_train.columns,vifs),key=lambda x: x[1], reverse=True)

[('InternetService_No', inf),
 ('OnlineSecurity_No internet service', inf),
 ('OnlineBackup_No internet service', inf),
 ('DeviceProtection_No internet service', inf),
 ('TechSupport_No internet service', inf),
 ('StreamingTV_No internet service', inf),
 ('StreamingMovies_No internet service', inf),
 ('PhoneService', 1795.2936129533036),
 ('MonthlyCharges', 879.4071653278096),
 ('InternetService_Fiber optic', 150.413015558894),
 ('MultipleLines_No phone service', 61.269312275148884),
 ('StreamingTV_Yes', 24.573123044650014),
 ('StreamingMovies_Yes', 24.400188765894438),
 ('TotalCharges', 10.871762430587555),
 ('tenure', 7.528269480250208),
 ('MultipleLines_Yes', 7.388728481143919),
 ('DeviceProtection_Yes', 6.990713871204519),
 ('OnlineBackup_Yes', 6.911783532524887),
 ('TechSupport_Yes', 6.498500323508462),
 ('OnlineSecurity_Yes', 6.478979774573932),
 ('Contract_Two year', 2.6312318158444175),
 ('PaymentMethod_Electronic check', 1.9381234819755606),
 ('PaymentMethod_Mailed check', 1.8

## Test high Vif dropping function

In [36]:
def drop_high_vif_features(X_train):
    '''drops columns with vif greater than 10
    '''
    finished=False
    while not finished:
        vifs=[variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
        high_vifs=sorted(zip (X_train.columns,vifs),key=lambda x: x[1], reverse=True)
        high_vif_col,high_vif_value=high_vifs[0]
        if high_vif_value>=10:
            print(f"dropping column {high_vif_col} with vif value of {high_vif_value:.1f}")
            
            X_train=X_train.drop(columns=[high_vif_col])
        else:
            print("Finished dropping columns")
            finished=True


    return X_train

In [31]:
X_train=drop_high_vif_features(X_train)

droppping columnInternetService_No with vif value of inf
droppping columnOnlineSecurity_No internet service with vif value of inf
droppping columnOnlineBackup_No internet service with vif value of inf
droppping columnDeviceProtection_No internet service with vif value of inf
droppping columnTechSupport_No internet service with vif value of inf
droppping columnStreamingTV_No internet service with vif value of inf
droppping columnPhoneService with vif value of 1795.3
droppping columnMonthlyCharges with vif value of 43.2
droppping columnTotalCharges with vif value of 20.0
Finished dropping columns


# Test loading training data without high vif values.

In [33]:
X_train,y_train=load_training_data(final=True)

In [35]:
X_train

,gender,SeniorCitizen,Partner,Dependents,tenure,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,0,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,0,0,7,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,0,0,1,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,0,0,29,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,0,0,0,3,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,0,0,1,0,1,1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5278,1,0,1,1,23,1,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
5279,0,0,1,1,12,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5280,0,1,0,0,12,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
